## Pandas 常用功能

首先一定要记住一点，Pnadas 底层就是 Numpy，因此也继承了 Numpy 的大部分逻辑，也就是说**一维 Series 就是列向量**！
一维向量的索引是「行」，因此 Pandas 的默认索引是行索引（`axis=0`）


>https://www.datacamp.com/cheat-sheet/pandas-cheat-sheet-for-data-science-in-python

![](./_img/pandas-cheat-sheet.webp)

In [125]:
import pandas as pd
import numpy as np

# 1. pandas 中的基础元素是 Series，即列向量。
## 每个 Series 的每一行都有一个「行标签」，标签默认为数字，但也可手动指定
## 如下的 s 是一个以字母为行标签的列向量
s = pd.Series([3, -5, 7, 4],  index=['a',  'b',  'c',  'd'])
s

a    3
b   -5
c    7
d    4
dtype: int64

In [49]:
## 使用数组方式或属性方式都能索引到对应位置的值
s[0] == s['a'] == s.a == 3

True

In [122]:
## Series 上的 Apply 与 Map

### 在 Series 的每一个元素上应用一个函数，返回一个新 Series，不是原地修改！
s.apply(lambda it: it > 1)

a     True
b    False
c     True
d     True
dtype: bool

In [151]:
# 2. 二维数组或者说表格，在 Pandas 中被称为数据帧 DataFrame
## 在 DataFrame 中，每一列是一个 Series，所有列都拥有同样的索引名称
data = {'Country': ['Belgium',  'India',  'Brazil'],

'Capital': ['Brussels',  'New Delhi',  'Brasilia'],

'Population': [11190846, 1303171035, 207847528]} 

# 通过 columns 指定了列的顺序（列名在 data 里已经包含了），通过 index 指定了行标签。
# 如果不设定行标签，默认使用数字。
df = pd.DataFrame(data,columns=['Country',  'Capital',  'Population'], index=['a', 'b', 'c'])
df

,Country,Capital,Population
a,Belgium,Brussels,11190846
b,India,New Delhi,1303171035
c,Brazil,Brasilia,207847528


In [149]:
### Pandas/Numpy 都是列操作优先，DataFrame 也默认仅支持按列索引
df['Country']  # 使用列标签进行索引，得到的是一个 Series 对象（列向量）

a    Belgium
b      India
c     Brazil
Name: Country, dtype: object

In [62]:
## loc/at 基于标签的 DataFrame 查询（我们知道行与列都拥有标签）
df.loc[:, ['Country']]  # 查询 country 这一列


,Country
a,Belgium
b,India
c,Brazil


In [53]:
df.loc[['b', 'c'], ['Country']]  # 查询 country 这一列的 b c 这两行

,Country
b,India
c,Brazil


In [68]:
### loc 是用于批量查询数据，而 at 是用于查询单个元素
df.at['a', 'Country'] == df.at[df.index[0], 'Country'] == 'Belgium'

True

In [87]:
## iloc/iat 是基于 index（数字）进行查询，而不是使用「标签」
df.iloc[:, [0]]  # 查询 country 这一列


,Country
a,Belgium
b,India
c,Brazil


In [80]:
df.iloc[1:, [0]]  # 查询 country 这一列从第二行起的所有行

,Country
b,India
c,Brazil


In [153]:
## 基于 bool 值进行批量索引
s.loc[~(s > 1)]  # 查询出所有 s <= 1 的行，符号 ~ 表示对 bool 值取反
s.loc[(s < -1) | (s > 2)]  # 支持使用 & | ^ 等符号进行按元素的 bool 值操作
df.loc[df['Population']>1200000000]  # 或者根据某一列的 bool 计算，过滤出需要的行

### 还可以通过 lambda 函数复用查询条件
xxx_selector = lambda it: it['Population'].notnull() & (it['Population'] > 1200000000)
df.loc[xxx_selector]

### 不推荐使用默认的索引方式！！！
s[~(s > 1)]  # 未使用 loc/iloc，这种方式会返回一个新 DataFrame！

b   -5
dtype: int64

In [111]:
## 修改 DataFrame 中的值

### 错误的赋值方式！Pandas 会弹出 Warning
### 未使用 loc/iloc，这样返回的是一个新 DataFrame，对它进行修改不会对 df 产生任何效果！
df[df['Population']>1200000000]['Population'] = -1  
df

<ipython-input-111-6e5547d2a707>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Population']>1200000000]['Population'] = -1


,Country,Capital,Population
a,Belgium,Brussels,11190846
b,India,New Delhi,1303171035
c,Brazil,Brasilia,207847528


In [112]:

df.loc[df['Population']>1200000000, ['Population']] = -1  # 批量赋值
df

,Country,Capital,Population
a,Belgium,Brussels,11190846
b,India,New Delhi,-1
c,Brazil,Brasilia,207847528


In [116]:
## 从数据集中删除某部分数据

### 返回不包含 a c 两行的新 DataFrame，不是原地删除！
new_s =s.drop(['a',  'c'])

### 返回不包含 Country 这一列的新 DataFrame，不是原地删除！
new_df = df.drop('Country', axis=1) 

### 原地删除 Country 这一列，返回 None
df.drop('Country', axis=1, inplace=True) == None

True

In [155]:
# 根据条件进行 drop，实际上是先查出符合条件的行的所有索引，然后再进行 drop
new_df = df.drop(df[df['Population']>1200000000].index)
new_df

,Country,Capital,Population
a,Belgium,Brussels,11190846
c,Brazil,Brasilia,207847528


In [170]:
# 在进行查询后，往往索引会乱掉
# 此函数直接将索引标签重置为递增整数，drop 表示彻底丢弃掉旧标签信息
new_df.reset_index(drop=True)

,Country,Capital,Population
0,Belgium,Brussels,11190846
1,Brazil,Brasilia,207847528


In [131]:
## DataFrame 上的 apply/applymap

### 也可通过 df.iterrows() 与 df.iteritems() 进行逐行/逐元素计算，但是这种遍历受 Python GIL 影响效率低下。

df = pd.DataFrame(np.arange(12).reshape((3,4)), columns=['a', 'b', 'c', 'd'])
df

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [139]:
### apply 默认应用在 DataFrame 的每一列上（列向量优先），比如对每一列的**所有行**求和
df.apply(lambda it: it.sum())

a    12
b    15
c    18
d    21
dtype: int64

In [140]:
### apply 当然也可以手动设定 axis，比如每一行的**所有列**求和
df.apply(lambda it: it.sum(), axis=1)

0     6
1    22
2    38
dtype: int64

In [138]:
### applymap 是应用在 DataFrame 的每一个元素上，比如
df.applymap(lambda it: it % 3 > 0)

,a,b,c,d
0,False,True,True,False
1,True,True,False,True
2,True,False,True,True


In [162]:
# DataFrame 的合并：concat join merge

df1 = pd.DataFrame(np.arange(12).reshape((3,4)), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.arange(13, 25).reshape((3,4)), columns=['a', 'b', 'c', 'd'])

# 列优先原则，默认是在 axis=0 方向进行拼接（相当于垂直堆叠）
pd.concat([df1, df2])

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
0,13,14,15,16
1,17,18,19,20
2,21,22,23,24


In [176]:
# 指定在 axis=1 方向进行拼接，也就是行向量方向，横向
df3 = pd.concat([df1, df2], axis=1)
df3

,a,b,c,d,a,b,c,d
0,0,1,2,3,13,14,15,16
1,4,5,6,7,17,18,19,20
2,8,9,10,11,21,22,23,24


In [186]:
# join 两个 DataFrame
# df1.join(df2)

# 使用 merge 按某一列的值进行横向拼接，类似 SQL 的 JOIN ON
## 默认以重叠列名当做链接键，默认 inner join
# df1.merge(df2)


In [ ]:
# 读取 csv，得到一个 DataFrame 对象
## 使用第一行作为 csv 列名
df = pd.read_csv("xxx.csv", header=0)
# csv 不包含列名，直接自动生成数字 0123 作为列名
df = pd.read_csv("xxx.csv", header=None)
## csv 不包含列名，手动设定列名
df = pd.read_csv("xxx.csv", names=['col1', 'col2', 'col3'])
# 尤其注意对 NULL 值的处理，这里自行设定要转换为 NULL 的值
df = pd.read_csv("xxx.csv", header=None, na_vaules=['NA', 'null', '', 'NAN'])

# 使用 python 中的数据构造 DataFrame 对象
## 方法一：使用字典，key 为列名，value 为列数据
df = pd.DataFrame({
    "col1": [1, 2, 3, 4],
    "col2": [2, 3, 4, 5],
    "col3": [3, 4, 5, 6],
})
## 方法二：使用两个列表
df = pd.DataFrame([
        [1, 2, 3, 4],
        [2, 3, 4, 5],
        [3, 4, 5, 6],
    ],
    names=['col1', 'col2', 'col3'],  # 或者直接省略 names，使用数字列名
)

# 导出 csv 文件，去掉索引，保留列名(header)
df.to_csv("xxx.csv", header=True, index=False)

# 查看表格的前五行
df.head()

# 转置此表格
df.T

# 排序
## axis=0 表示按行排序，这也是默认行为
df.sort_values(['col1', 'col2'], axis=0, ascending=True)  # 先使用 col1 排序，col1 不能确定的再按 col2 排序
## axis=1 或者 axis='columns' 表示按列排序

# 修改 DataFrame 的 index 或者 column names
df.rename(columns=['c1', 'c2', 'c3'])

# 使用 groupby 进行分组，对 col3 这一列进行分组求和（感觉和 SQL 有点像）
df.groupby(['col1', 'col2'])['col3'].sum()
